In [3]:

# Imports

import pandas as pd
import re
import html
from bs4 import BeautifulSoup


In [4]:

QUESTIONS_PATH = "../data/raw/Questions.csv"
ANSWERS_PATH = "../data/raw/Answers.csv"
TAGS_PATH = "../data/raw/Tags.csv"

OUTPUT_PATH = "../data/processed/questions_clean.csv"

CHUNK_SIZE = 50000  # rows processed at a time(large so chunckwise)


counting questions/answers/tags

In [5]:

question_count = sum(
    len(chunk)
    for chunk in pd.read_csv(QUESTIONS_PATH, encoding="latin1", chunksize=100000)
)

question_count


1264216

In [6]:
answer_count = sum(
    len(chunk)
    for chunk in pd.read_csv(ANSWERS_PATH, encoding="latin1", chunksize=100000)
)

answer_count


2014516

In [7]:
tag_count = sum(
    len(chunk)
    for chunk in pd.read_csv(TAGS_PATH, encoding="latin1", chunksize=100000)
)

tag_count


3750994

In [8]:
dataset_summary = pd.DataFrame({
    "TYpe": ["Questions", "Answers", "Tags"],
    "Count": [question_count, answer_count, tag_count]
})

dataset_summary


,TYpe,Count
0,Questions,1264216
1,Answers,2014516
2,Tags,3750994


In [19]:
sample_df_ques= pd.read_csv(
    QUESTIONS_PATH,
    encoding="latin1",
    nrows=5
)

# sample_df_ques.head()
sample_df_ques[['Title','Body']].head()


,Title,Body
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [16]:
sample_df_ques= pd.read_csv(
    QUESTIONS_PATH,
    encoding="latin1",
    nrows=5
)

sample_df_ques.shape


(5, 7)

In [10]:
def clean_text_pipeline(text):
    if pd.isna(text):
        return ""

    # Decode HTML entities (e.g., &lt; → <)
    text = html.unescape(text)

    # Remove HTML/XML tags
    text = BeautifulSoup(text, "lxml").get_text()

    # Convert to lowercase
    text = text.lower()

    # Remove non-alphabetic characters
    text = re.sub(r"[^a-z\s]", " ", text)

    # Normalize multiple spaces
    text = re.sub(r"\s+", " ", text)

    return text.strip()


In [11]:
# Test Cleaning on Small Subset
test_df = pd.read_csv(
    QUESTIONS_PATH,
    encoding="latin1",
    nrows=10000
)

test_df["raw_text"] = (
    test_df["Title"].fillna("") + " " +
    test_df["Body"].fillna("")
)

test_df["clean_text"] = test_df["raw_text"].apply(clean_text_pipeline)

sample = test_df[["raw_text", "clean_text"]].iloc[0]

print("RAW TEXT:\n", sample["raw_text"][:500])
print("\nCLEAN TEXT:\n", sample["clean_text"][:500])


RAW TEXT:
 SQLStatement.execute() - multiple queries in one statement <p>I've written a database generation script in <a href="http://en.wikipedia.org/wiki/SQL">SQL</a> and want to execute it in my <a href="http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime">Adobe AIR</a> application:</p>

<pre><code>Create Table tRole (
      roleID integer Primary Key
      ,roleName varchar(40)
);
Create Table tFile (
    fileID integer Primary Key
    ,fileName varchar(50)
    ,fileDescription varchar(500)
    ,thum

CLEAN TEXT:
 sqlstatement execute multiple queries in one statement i ve written a database generation script in sql and want to execute it in my adobe air application create table trole roleid integer primary key rolename varchar create table tfile fileid integer primary key filename varchar filedescription varchar thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isne

In [12]:
# Full Dataset Cleaning (Chunk Processing)
first_chunk = True

for chunk in pd.read_csv(
    QUESTIONS_PATH,
    encoding="latin1",
    chunksize=CHUNK_SIZE
):

    # Combine title and body
    chunk["raw_text"] = (
        chunk["Title"].fillna("") + " " +
        chunk["Body"].fillna("")
    )

    # Apply cleaning pipeline
    chunk["clean_text"] = chunk["raw_text"].apply(clean_text_pipeline)

    # Keep only relevant columns
    processed_chunk = chunk[["Id", "Score", "clean_text"]]

    # Write incrementally
    processed_chunk.to_csv(
        OUTPUT_PATH,
        mode="w" if first_chunk else "a",
        header=first_chunk,
        index=False
    )

    first_chunk = False

print(" Full preprocessing complete.")


 Full preprocessing complete.


In [13]:
# verify
clean_df_sample = pd.read_csv(OUTPUT_PATH, nrows=5)

clean_df_sample.head()


,Id,Score,clean_text
0,80,26,sqlstatement execute multiple queries in one s...
1,90,144,good branching and merging tutorials for torto...
2,120,21,asp net site maps has anyone got experience cr...
3,180,53,function for creating color wheels this is som...
4,260,49,adding scripting functionality to net applicat...


In [14]:
original_count = sum(
    len(chunk)
    for chunk in pd.read_csv(QUESTIONS_PATH, encoding="latin1", chunksize=100000)
)

clean_count = sum(
    len(chunk)
    for chunk in pd.read_csv(OUTPUT_PATH, chunksize=100000)
)

print("Original rows:", original_count)
print("Cleaned rows:", clean_count)


Original rows: 1264216
Cleaned rows: 2214216


answers cleaning

In [21]:
sample_df_ans= pd.read_csv(
    ANSWERS_PATH,
    encoding="latin1",
    nrows=5
)

# sample_df_ans.head()
sample_df_ques[['Body']].head()


,Body
0,<p>I've written a database generation script i...
1,<p>Are there any really good tutorials explain...
2,<p>Has anyone got experience creating <strong>...
3,<p>This is something I've pseudo-solved many t...
4,<p>I have a little game written in C#. It uses...


In [ ]:
# Test Cleaning on Small Subset of ans
test_df_ans = pd.read_csv(
    ANSWERS_PATH,
    encoding="latin1",
    nrows=10000
)

test_df_ans["raw_text"] = (
    test_df_ans["Body"].fillna("")
)

test_df_ans["clean_text"] = test_df_ans["raw_text"].apply(clean_text_pipeline)

sample = test_df_ans[["raw_text", "clean_text"]].iloc[0]

print("RAW TEXT:\n", sample["raw_text"][:500])
print("\nCLEAN TEXT:\n", sample["clean_text"][:500])


RAW TEXT:
 <p><a href="http://svnbook.red-bean.com/">Version Control with Subversion</a></p>

<p>A very good resource for source control in general. Not really TortoiseSVN specific, though.</p>

CLEAN TEXT:
 version control with subversion a very good resource for source control in general not really tortoisesvn specific though


In [23]:
# Full Dataset Cleaning (Chunk Processing)
first_chunk = True

for chunk in pd.read_csv(
    ANSWERS_PATH,
    encoding="latin1",
    chunksize=CHUNK_SIZE
):

    # Combine title and body
    chunk["raw_text"] = (
        chunk["Body"].fillna("")
    )

    # Apply cleaning pipeline
    chunk["clean_text"] = chunk["raw_text"].apply(clean_text_pipeline)

    # Keep only relevant columns
    processed_chunk = chunk[["Id", "Score", "clean_text"]]

    # Write incrementally
    processed_chunk.to_csv(
        OUTPUT_PATH,
        mode="w" if first_chunk else "a",
        header=first_chunk,
        index=False
    )

    first_chunk = False

print(" Full preprocessing complete.")


 Full preprocessing complete.


In [24]:
# verify
clean_df_sample = pd.read_csv(OUTPUT_PATH, nrows=5)

clean_df_sample.head()


,Id,Score,clean_text
0,92,13,version control with subversion a very good re...
1,124,12,i wound up using this it is a kind of a hack b...
2,199,1,i ve read somewhere the human eye can t distin...
3,269,4,yes i thought about that but i soon figured ou...
4,307,28,oleg shilo s c script solution at the code pro...


Original rows: 2014516
Cleaned rows: 2014516
